In [3]:
! pip install git+https://github.com/openai/whisper.git -q

In [4]:
import whisper
model = whisper.load_model("large") #769M Parameter

100%|██████████████████████████████████████| 2.88G/2.88G [00:16<00:00, 183MiB/s]


In [5]:
!pip install SpeechRecognition
!pip install ffmpeg-python
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 43.6 MB/s eta 0:00:00:00:0100:01


# Extracting and adding all audio files to output folder

In [8]:
import os
import shutil

def extract_audios(folder_path, output_folder):
    """
    Recursively extracts all audio files from the given folder path
    and copies them to the output folder.
    """
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            if file_path.endswith(('.mp3', '.wav', '.flac', '.ogg', '.aac')):  # Add more audio extensions if needed
                output_file_path = os.path.join(output_folder, file)
                os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
                shutil.copy(file_path, output_file_path)  # Copy the audio file using shutil.copy

# Example usage
audio_folder_path = '/kaggle/input/hindi-recogniniton/audio'
output_folder_path = '/kaggle/working/output'
extract_audios(audio_folder_path, output_folder_path)

# Generating the hypothesis and adding to df as "transcript"

In [9]:
import os
import pandas as pd
import speech_recognition as sr

def generate_transcript(file_path):
    r = sr.Recognizer()
    with sr.AudioFile(file_path) as source:
        audio = r.record(source)
    try:
        result = r.recognize_google(audio, language='hi')
        return result
    except sr.UnknownValueError:
        print(f"Speech recognition could not understand audio: {file_path}")
    except sr.RequestError as e:
        print(f"Could not request results from Speech Recognition service; {e}")

audio_folder_path = '/kaggle/working/output'
data = []

for root, dirs, files in os.walk(audio_folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        if file_path.endswith(('.mp3', '.wav', '.flac', '.ogg', '.aac')):
            transcript = generate_transcript(file_path)
            data.append({'audio_file': file, 'transcript': transcript})

df = pd.DataFrame(data, columns=['audio_file', 'transcript'])
print(df)

                     audio_file  \
0     844424932563110-769-m.wav   
1     844424930898390-574-f.wav   
2     844424932410099-267-f.wav   
3     844424933481734-288-m.wav   
4     844424933454662-288-m.wav   
...                         ...   
1924  844424930927720-574-f.wav   
1925  844424930547238-497-f.wav   
1926  844424932563111-769-m.wav   
1927  844424930898438-574-f.wav   
1928  844424933474163-540-m.wav   

                                             transcript  
0     सेनापति की सजा सामान माहिया किराने की दिशा में...  
1     कारोबारी रंग के मामले में भारत की स्थिति उतनी ...  
2     आदिकाल से ही यह हर बार गर्मी के मौसम में लग जा...  
3           इसके अतिरिक्त आंख की एक और जाति पाई जाती है  
4              लोग इस आंतरिक कूट के नाम से भी जानते हैं  
...                                                 ...  
1924  महिला एवं शेरपा को प्रधान तथा टाइल शेरपा को सं...  
1925  अधिकारियों ने बताया कि यह दीवार बीड़ी पियो द्व...  
1926    मैं अब भारत में इसके शुरू होने का इंतजार कर रही  

# Importing the file having groundtruths which has to be compared to the generated transcripts.

In [ ]:
gt = pd.read_csv("/kaggle/input/test-dataset/bucket.csv")

# Adding the groundtruth text to df 

In [21]:
def get_ground_truth(audio_file):
    # Check if the audio_file is present in the 'file_path' column of gt
    matched = gt['file_path'].str.contains(audio_file, case=False)
    
    if matched.any():
#         print('matched')
        # Get the 'ground truth' value for the matched row
        return gt.loc[matched, 'transcript'].values[0]
    else:
        return ''

# Apply the function to the 'audio_file' column and create a new 'ground_truth' column
df['ground_truth'] = df['audio_file'].apply(get_ground_truth)



In [22]:
df.head(5)

,audio_file,transcript,ground_truth
0,844424932563110-769-m.wav,सेनापति की सजा सामान माहिया किराने की दिशा में...,सेना को आधुनिक साजोसामान मुहैया कराने की दिशा ...
1,844424930898390-574-f.wav,कारोबारी रंग के मामले में भारत की स्थिति उतनी ...,कारोबारी ऋण के मामले में भारत की स्थिति उतनी ब...
2,844424932410099-267-f.wav,आदिकाल से ही यह हर बार गर्मी के मौसम में लग जा...,आदिकाल से ही यह हर बार गर्मी के मौसम में लग जा...
3,844424933481734-288-m.wav,इसके अतिरिक्त आंख की एक और जाति पाई जाती है,इसके अतिरिक्त आक की एक और जाति पाई जाती है
4,844424933454662-288-m.wav,लोग इस आंतरिक कूट के नाम से भी जानते हैं,लोग इसे अन्नकूट के नाम से भी जानते हैं


# calculating WER from the given transcript and ground_truth

In [26]:
import pandas as pd
from jiwer import wer

def calculate_wer(row):
    if pd.isna(row['ground_truth']):
        return float('nan')
    else:
        transcript = row['transcript']
        ground_truth = row['ground_truth']
        return wer(ground_truth, transcript)

df['wer'] = df.apply(calculate_wer, axis=1)


In [27]:
df.head(5)

,audio_file,transcript,ground_truth,wer
0,844424932563110-769-m.wav,सेनापति की सजा सामान माहिया किराने की दिशा में...,सेना को आधुनिक साजोसामान मुहैया कराने की दिशा ...,0.571429
1,844424930898390-574-f.wav,कारोबारी रंग के मामले में भारत की स्थिति उतनी ...,कारोबारी ऋण के मामले में भारत की स्थिति उतनी ब...,0.083333
2,844424932410099-267-f.wav,आदिकाल से ही यह हर बार गर्मी के मौसम में लग जा...,आदिकाल से ही यह हर बार गर्मी के मौसम में लग जा...,0.000000
3,844424933481734-288-m.wav,इसके अतिरिक्त आंख की एक और जाति पाई जाती है,इसके अतिरिक्त आक की एक और जाति पाई जाती है,0.100000
4,844424933454662-288-m.wav,लोग इस आंतरिक कूट के नाम से भी जानते हैं,लोग इसे अन्नकूट के नाम से भी जानते हैं,0.333333
